In [13]:
import os
import copy

import numpy as np
import torch
import torch.nn as nn
import math

from torch import optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from unet import UNet
from dice_loss import dice_coeff
####################################################
# for data splitting
####################################################
import pandas as pd
####################################################
# for data preparation
####################################################
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score, accuracy_score, balanced_accuracy_score
####################################################
# for plotting
####################################################
import matplotlib.pyplot as plt
from IPython.display import clear_output
import random
############################
# Helper func
############################
from helper import * 

########################################
N_CHANNELS = 1 #greyscale
N_CLASSES = 1 # classes, IRF, SRF, PED

BATCH_SIZE, EPOCHS = 16, 150
IMAGE_SIZE = (224, 224)
CROP_SIZE = (200, 200)

In [14]:
PTH = r'C:\Users\utente\Desktop\Università\Tesi magistrale\FedMix\data\labelsTrBW'
PTH_IM = r'C:\Users\utente\Desktop\Università\Tesi magistrale\FedMix\data\imagesTrAug'

miccai_ = np.arange(1,200)
bns_ = np.arange(200,380)

miccai_test = np.random.choice(miccai_, size=5, replace=False)
miccai_test = [str(a) for a in miccai_test]
miccai_train = [str(a) for a in miccai_ if str(a) not in miccai_test]
print(len(miccai_train)/len(miccai_))

bns_test = np.random.choice(bns_, size=5, replace=False)
bns_test = [str(a) for a in bns_test]
bns_train = [str(a) for a in bns_ if str(a) not in bns_test]
print(len(bns_train)/len(bns_))

whole_data =  os.listdir(PTH)

0.9748743718592965
0.9722222222222222


In [15]:
TEST_SPLIT = dict()
TEST_SPLIT['miccai'] = miccai_test
TEST_SPLIT['bns'] = bns_test

In [16]:
WHOLE_DATA_TRAIN = dict()
WHOLE_DATA_TEST = dict()

WHOLE_DATA_TRAIN['miccai'] = []
WHOLE_DATA_TRAIN['bns'] = []

WHOLE_DATA_TEST['miccai'] = []
WHOLE_DATA_TEST['bns'] = []

import random

# Calcola il numero totale di campioni
total_samples = len(whole_data)

# Calcola il numero di campioni da utilizzare per il test
test_size = int(0.1 * total_samples)

# Estrai casualmente un subset di campioni per il test
test_samples = random.sample(whole_data, test_size)

# Itera attraverso i dati e assegna le immagini e le label al set di test se sono presenti nei campioni selezionati
for item in whole_data:
    separator = item.split('_') # Identifica la fonte
    sample_number = separator[1].split('0')[-1]
    
    # Controlla se l'elemento corrente è presente nei campioni di test
    if item in test_samples:
        # Aggiungi l'elemento al set di test
        data_path = (PTH_IM+'\\'+item, PTH+'\\'+item) # X,Y source
        WHOLE_DATA_TEST[separator[0]].append(data_path)
    else:
        # Aggiungi l'elemento al set di allenamento
        data_path = (PTH_IM+'\\'+item, PTH+'\\'+item) # X,Y source
        WHOLE_DATA_TRAIN[separator[0]].append(data_path)



In [17]:
LEN_bns = 0
print('name\t\t train\t test')
for item in WHOLE_DATA_TRAIN:
    if item == 'bns':
        print(item,'\t', len(WHOLE_DATA_TRAIN[item]),'\t', len(WHOLE_DATA_TEST[item]))
        LEN_bns = WHOLE_DATA_TRAIN[item]
    else:
        print(item,'\t\t', len(WHOLE_DATA_TRAIN[item]),'\t', len(WHOLE_DATA_TEST[item]))

name		 train	 test
miccai 		 181 	 19
bns 	 323 	 37


In [18]:
device = torch.device('cuda:0')

WEIGHTS_CL = [0.0,0.0]
CLIENTS = ['miccai', 'bns']
CLIENTS_2 = [cl +'_2' for cl in CLIENTS]
TOTAL_CLIENTS = len(CLIENTS)

LR = 1.5e-3
WD = 1e-5
TH = 0.9

LAMBDA_ =2
BETA_=3
TH = 0.9

for idx, client in enumerate(WHOLE_DATA_TRAIN):
    WEIGHTS_CL[idx] = len(WHOLE_DATA_TRAIN[client])

    
total_weight = sum(WEIGHTS_CL)
WEIGHTS_CL = [s/total_weight for s in WEIGHTS_CL]

In [19]:
CLIENTS_SUPERVISION = ['labeled', 'labeled']

In [20]:
split_dataset = dict()
for cl in CLIENTS:
    split_dataset[cl+'_train'] = retouch(WHOLE_DATA_TRAIN[cl], train=True)
    split_dataset[cl+'_test'] = retouch(WHOLE_DATA_TEST[cl], train=False)

In [21]:
TOTAL_CLIENTS = 2
GLOBAL_ACC = 0.0

training_clients, testing_clients = dict(), dict()
########## aditional #####################
training_clients_pl = dict()
acc_train, acc_test, loss_train, loss_test = dict(), dict(), \
                                            dict(), dict()

nets, optimizers = dict(), dict()

nets['global'] = UNet(n_channels=N_CHANNELS, n_classes=N_CLASSES, \
                      bilinear=True).to(device)

nets['global_2'] = UNet(n_channels=N_CHANNELS, n_classes=N_CLASSES, \
                      bilinear=True).to(device)

for client, c_sup in zip(CLIENTS, CLIENTS_SUPERVISION):
    if c_sup == 'labeled':
        training_clients[client] = DataLoader(split_dataset[client+'_train'], batch_size=16,\
                     shuffle=True, num_workers=8)
        training_clients_pl[client] = DataLoader(split_dataset[client+'_train'], batch_size=1, \
                    shuffle=True, num_workers=8)
    else:
        training_clients[client] = DataLoader(split_dataset[client+'_train'], batch_size=16,\
                             shuffle=True, num_workers=8)
        ################# additional dataloader ##########################################
        training_clients_pl[client] = DataLoader(split_dataset[client+'_train'], batch_size=1,\
                             shuffle=True, num_workers=8)
        training_clients_pl[client+'_2'] = DataLoader(split_dataset[client+'_train'], batch_size=1,\
                             shuffle=True, num_workers=8)
    ###################################################################################
    testing_clients[client] = DataLoader(split_dataset[client+'_test'], batch_size=16,\
                         shuffle=False, num_workers=1)
    
    acc_train[client], acc_test[client] = [], []
    loss_train[client], loss_test[client] = [], []
        
    nets[client] = UNet(n_channels=N_CHANNELS, n_classes=N_CLASSES, \
                      bilinear=True).to(device)
    nets[client+'_2'] = UNet(n_channels=N_CHANNELS, n_classes=N_CLASSES, \
                      bilinear=True).to(device)
    optimizers[client]= optim.Adam(nets[client].parameters(), \
                                   lr=LR,weight_decay=WD)
    optimizers[client+'_2']= optim.Adam(nets[client+'_2'].parameters(), \
                                   lr=LR,weight_decay=WD)

In [22]:
torch.multiprocessing.set_sharing_strategy('file_system')

In [23]:
WEIGHTS_CL

[0.35912698412698413, 0.6408730158730159]

In [24]:
best_avg_acc, best_epoch_avg = 0,0
index = []

for client in CLIENTS:
    acc_train[client], acc_test[client] = [], []
    loss_train[client], loss_test[client] = [], [] 
    
for epoch in range(EPOCHS):
    index.append(epoch)
    ####### conduct training #####
    #################### copy fed model ###################
    copy_fed(CLIENTS, nets, fed_name='global')
    
    for client, supervision_t in zip(CLIENTS, CLIENTS_SUPERVISION):
        print(client)
        train_model(training_clients[client], nets[client], \
                                  optimizers[client], device, \
                                  acc = acc_train[client], \
                                  loss = loss_train[client], \
                                  supervision_type = supervision_t)
        
    aggr_fed(CLIENTS, WEIGHTS_CL, nets, fed_name='global')
    ################### test ##############################
    avg_acc = 0.0
    for client in CLIENTS:
        test(epoch, testing_clients[client], nets[client], device, acc_test[client],\
             loss_test[client])
        avg_acc += acc_test[client][-1]
        
    avg_acc = avg_acc / TOTAL_CLIENTS
    ############################################################
    ########################################################
    if avg_acc > best_avg_acc:
        best_avg_acc = avg_acc
        best_epoch = epoch
    
    ################################
    # plot #########################
    ################################
    clear_output(wait=True)
    print(avg_acc, best_avg_acc)
    plot_graphs(0, CLIENTS, index, acc_train, 'acc_train')
    plot_graphs(1, CLIENTS, index, loss_train, 'loss_train')
    plot_graphs(2, CLIENTS, index, acc_test, ' acc_test')

print(best_avg_acc, best_epoch)
for client in CLIENTS:
    print(client)
    tmp = best_epoch
    best_epoch = best_epoch 
    print("shared epoch specific")
    print(acc_test[client][best_epoch])
    print("max client-specific")
    print(np.max(acc_test[client]))
    best_epoch = tmp

miccai


ValueError: too many values to unpack (expected 3)

In [25]:
# Stampa un batch completo
for batch_idx, batch in enumerate(training_clients[client]):
    print("Batch:", batch_idx + 1)
    for data in batch:
        print("Forma del tensore:", data.shape)


Batch: 1
Forma del tensore: torch.Size([16, 1, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Batch: 2
Forma del tensore: torch.Size([16, 1, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Batch: 3
Forma del tensore: torch.Size([16, 1, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Batch: 4
Forma del tensore: torch.Size([16, 1, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Batch: 5
Forma del tensore: torch.Size([16, 1, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Forma del tensore: torch.Size([16, 3, 200, 200])
Forma del tensore: torch